In [1]:
import os
# import zipfile
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = (SparkSession
     .builder
     .master('local[*]') # Master URL;
     .getOrCreate())

In [3]:
from pyspark.sql.types import (StructType, StructField, StringType,
IntegerType, FloatType, TimestampType, LongType)
import glob

# Define the schema—NOT FINAL; SHOULD REVISIT
schema = StructType([
    StructField("GKGRECORDID", StringType(), True),
    StructField("V2.1DATE", LongType(), True),
    StructField("V2SOURCECOLLECTIONIDENTIFIER", IntegerType(), True),
    StructField("V2SOURCECOMMONNAME", StringType(), True),
    StructField("V2DOCUMENTIDENTIFIER", StringType(), True),
    StructField("V1COUNTS", StringType(), True),
    StructField("V2.1COUNTS", StringType(), True),
    StructField("V1THEMES", StringType(), True),
    StructField("V2ENHANCEDTHEMES", StringType(), True),
    StructField("V1LOCATIONS", StringType(), True),
    StructField("V2ENHANCEDLOCATIONS", StringType(), True),
    StructField("V1PERSONS", StringType(), True),
    StructField("V2ENHANCEDPERSONS", StringType(), True),
    StructField("V1ORGANIZATIONS", StringType(), True),
    StructField("V2ENHANCEDORGANIZATIONS", StringType(), True),
    StructField("V1.5TONE", FloatType(), True),
    StructField("V2.1ENHANCEDDATES", StringType(), True),
    StructField("V2GCAM", StringType(), True),
    StructField("V2.1SHARINGIMAGE", StringType(), True),
    StructField("V2.1RELATEDIMAGES", StringType(), True),
    StructField("V2.1SOCIALIMAGEEMBEDS", StringType(), True),
    StructField("V2.1SOCIALVIDEOEMBEDS", StringType(), True),
    StructField("V2.1QUOTATIONS", StringType(), True),
    StructField("V2.1ALLNAMES", StringType(), True),
    StructField("V2.1AMOUNTS", StringType(), True),
    StructField("V2.1TRANSLATIONINFO", StringType(), True),
    StructField("V2EXTRASXML", StringType(), True)
])

df_gkg = spark.read.csv('/mnt/data/public/gdeltv2/gkg/201908*.gkg.csv',
                        sep='\t', schema=schema)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3a98d89290>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterr

KeyboardInterrupt: 

In [ ]:
df_gkg.limit(3).toPandas()

In [ ]:
(df_gkg
 .withColumn('LOCATION', F.regexp_extract('V1COUNTS',r",\s*([^,#]+)#", 1))
 .filter(F.col('LOCATION').rlike('Philippines'))
 .count()
)

In [ ]:
import os
import datetime

def calculate_and_print_total_size_per_week_of_august(directory, year):
    """
    Calculate and print the total file size of files for each week of August of a specified year.

    Parameters:
        directory (str): The directory containing the files.
        year (int): The year for which files are considered for the month of August.

    Returns:
        None: Prints the total size per week directly.
    """
    # Define year as a string for matching
    year_str = str(year)
    
    # Initialize dictionary to hold file sizes for each week of August
    weekly_sizes = {week: 0 for week in range(1, 6)}

    # List files in the directory
    try:
        files = os.listdir(directory)
    except FileNotFoundError:
        print(f"Directory not found: {directory}")
        return

    # Filter and accumulate file sizes for each week of August
    for file in files:
        if file.endswith('.gkg.csv') and file.startswith(year_str + '08'):
            # Extract the date from the filename (assuming format YYYYMMDD)
            date_str = file[:8]  # YYYYMMDD
            date_obj = datetime.datetime.strptime(date_str, '%Y%m%d')
            week_of_month = (date_obj.day - 1) // 7 + 1
            
            # Construct the full file path
            file_path = os.path.join(directory, file)
            # Add the file size in bytes to the total for the week
            weekly_sizes[week_of_month] += os.stat(file_path).st_size

    # Print the results for each week
    for week in range(1, 6):
        total_size_bytes = weekly_sizes[week]
        total_size_mb = total_size_bytes / (1024**2)
        total_size_gb = total_size_bytes / (1024**3)
        print(f"Total size for {year}-08, week {week}: {total_size_bytes} bytes, {total_size_mb:.2f} MB, {total_size_gb:.2f} GB")

# Example usage
directory = '/mnt/data/public/gdeltv2/gkg'
year = 2019
calculate_and_print_total_size_per_week_of_august(directory, year)